In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from sklearn.model_selection import train_test_split
import warnings
import pandas as pd
import numpy as np
import helpers
import os
import random

warnings.filterwarnings("ignore")

In [2]:
class positionalEncoder(nn.Module):

  def __init__(self, frame_length, encoding_length):
    super().__init__()

    self.embedding = nn.Embedding(frame_length, encoding_length)

    self.frame_length = frame_length
    
    
  def forward(self, x):

    pe = self.embedding(torch.tensor([i for i in range(self.frame_length)]))

    if len(x.shape) == 3:
      if len(pe.shape) != 3:
        pe = pe.unsqueeze(0).repeat(x.shape[0], 1, 1)
      
      x = torch.cat((x, pe[:x.shape[0]]), 2)
    else:
      x = torch.cat((x, pe[:x.shape[0]]), 1)

    return x

In [3]:
class classifierTransformer(nn.Module):

  def __init__(self, inFeatCount, num_T_layers, num_frames, device, pos_encode_size = 5, n_heads = 4, n_hidden = 2048, dropout = 0.3, outFeatCount = 2):
    super().__init__()

    self.posEncoder = positionalEncoder(num_frames, pos_encode_size)

    heads = n_heads
    num_features = inFeatCount + pos_encode_size

    if (num_features % heads) != 0:
      heads += heads - (num_features % heads)

    print(f'features = {num_features}, heads = {heads}')

    n_hidden = max(n_hidden, 2*num_features)

    encoder_layer = nn.TransformerEncoderLayer(inFeatCount + pos_encode_size, heads, n_hidden, dropout)
    self.encoder = nn.TransformerEncoder(encoder_layer, num_T_layers)
    
    many_to_one_feat = num_frames * num_features
    mid = (many_to_one_feat - outFeatCount) // 2 + outFeatCount

    self.fc1 = nn.Linear(many_to_one_feat, mid)
    self.fc2 = nn.Linear(mid, 2)

    self.device = device

    self.init_weights()

  def init_weights(self):
      initrange = 0.1
      self.fc1.bias.data.zero_()
      self.fc1.weight.data.uniform_(-initrange, initrange)

      self.fc2.bias.data.zero_()
      self.fc2.weight.data.uniform_(-initrange, initrange)

  def forward(self, x):
    
    #x.shape = [num_frames, feat_count]
    encoded = self.posEncoder(x)

    #encoded.shape = [num_frames, feat_count + pos_encoding_count]
    data = self.encoder(encoded)

    #data.shape = [num_frames, feat_count + pos_encoding_count]
    if len(data.shape) == 3:
      data = torch.reshape(data, (data.shape[0], data.shape[1] * data.shape[2]))
    else:
      data = torch.reshape(data, (1,-1))

    #data.shape = [1, num_frames * (feat_count + pos_encoding_count)] 

    data = self.fc1(data)
    data = self.fc2(data)
    ##data = nn.functional.softmax(data, dim = 1).to(self.device) 

    return data.float()

In [ ]:
lie_trial_path = './data/OpenFace/trial/lie/' #60 entries
truth_trial_path = './data/OpenFace/trial/truth/' #61 entries

lie_MU3D_path = './data/OpenFace/MU3D/lie/' 
truth_MU3D_path = './data/OpenFace/MU3D/truth/'

lie_BOL_path = './data/OpenFace/BOL/lie/' 
truth_BOL_path = './data/OpenFace/BOL/truth/' 

#OFTruth = [truth_trial_path, truth_MU3D_path, truth_BOL_path]
#OFLie = [lie_trial_path, lie_MU3D_path, lie_BOL_path]

OFTruth = [truth_trial_path]
OFLie = [lie_trial_path]

lie_trial_path = './data/TransFormer/trial/lie/' #60 entries
truth_trial_path = './data/TransFormer/trial/truth/' #61 entries

lie_MU3D_path = './data/TransFormer/MU3D/lie/'
truth_MU3D_path = './data/TransFormer/MU3D/truth/'

lie_BOL_path = './data/TransFormer/BOL/lie/'
truth_BOL_path = './data/TransFormer/BOL/truth/'

#TTruth = [truth_trial_path, truth_MU3D_path, truth_BOL_path]
#TLie = [lie_trial_path, lie_MU3D_path, lie_BOL_path]

TTruth = [truth_trial_path]
TLie = [lie_trial_path]

features = ["gaze_0_x","gaze_0_y","gaze_0_z","gaze_angle_x", "gaze_angle_y", "AU01_r","AU04_r","AU10_r","AU12_r","AU45_r"]

# 0 = Openface
# 1 = Openface + TransFormer
# 2 = TransFormer

mode = 0

def processingOF(truthPath, liePath, minConfidence = 0.9, numOfFrames = 10):

    data = []
    label = []

    #truthPath is going to be a list of paths
    for path in truthPath:
        for file in sorted(os.listdir(path)):
            if file.endswith(".csv"):
                df = pd.read_csv(path + file)
            
                truth_bad_frame = set(np.where(df["confidence"] < minConfidence)[0])
                df = helpers.filterColumn(df, colList=features)

                index = numOfFrames
                next_index = numOfFrames
                
                while index < len(df):
                    if index not in truth_bad_frame and index >= next_index:
                        data.append((df.iloc[index-numOfFrames:index]).to_numpy())
                        label.append(1)
                    elif index in truth_bad_frame:
                        next_index = index + numOfFrames
                    index += 1

    for path in liePath:
        for file in sorted(os.listdir(path)):
            if file.endswith(".csv"):
                df = pd.read_csv(path + file)
            
                lie_bad_frame = set(np.where(df["confidence"] < minConfidence)[0])
                df = helpers.filterColumn(df, colList=features)

                index = numOfFrames
                next_index = numOfFrames
                
                while index < len(df):
                    if index not in lie_bad_frame and index >= next_index:
                        data.append((df.iloc[index-numOfFrames:index]).to_numpy())
                        label.append(0)
                    elif index in lie_bad_frame:
                        next_index = index + numOfFrames
                    index += 1

    data = np.array(data)
    label = np.array(label)
    random.seed(random.randint(1, 100))

    # Create an array of indices, then shuffle it
    indices = np.arange(len(data)).astype(int)
    np.random.shuffle(indices)

    # Same order of indices for both X and Y
    data  = data[indices]
    label = label[indices]

    return data, label, truth_bad_frame, lie_bad_frame

def processingTF(truthPath, liePath, numOfFrames = 10, combine = False, truth_bad_frame = None, lie_bad_frame = None):
                
        data = []
        label = []
    
        #truthPath is going to be a list of paths
        for path in truthPath:
            for file in sorted(os.listdir(path)):
                if file.endswith(".csv"):
                    df = pd.read_csv(path + file)
    
                    index = numOfFrames
                    next_index = numOfFrames
                    
                    while index < len(df):

                        if combine:
                            if index not in truth_bad_frame and index >= next_index:
                                data.append((df.iloc[index-numOfFrames:index]).to_numpy())
                                label.append(1)
                            else:
                                next_index = index + numOfFrames
                        else:
                            data.append((df.iloc[index-numOfFrames:index]).to_numpy())
                            label.append(1)
                        
                        index += 1
    
        for path in liePath:
            for file in sorted(os.listdir(path)):
                if file.endswith(".csv"):
                    df = pd.read_csv(path + file)
    
                    index = numOfFrames
                    next_index = numOfFrames
                    
                    if combine:
                        if index not in lie_bad_frame and index >= next_index:
                            data.append((df.iloc[index-numOfFrames:index]).to_numpy())
                            label.append(0)
                        else:
                            next_index = index + numOfFrames
                    else:
                        data.append((df.iloc[index-numOfFrames:index]).to_numpy())
                        label.append(0)
                    
                    index += 1
    
        data = np.array(data)
        label = np.array(label)
        random.seed(random.randint(1, 100))
    
        # Create an array of indices, then shuffle it
        indices = np.arange(len(data)).astype(int)
        np.random.shuffle(indices)
    
        # Same order of indices for both X and Y
        data  = data[indices]
        label = label[indices]
    
        if combine:
            data = np.reshape(data, (data.shape[0], data.shape[1] * data.shape[2]))
    
        return data, label

if mode == 0:
    X, Y, _, _ = processingOF(OFTruth, OFLie)
elif mode == 1:
    X1, Y, TB, LB = processingOF(OFTruth, OFLie)
    X2, _ = processingTF(TTruth, TLie, combine = True, truth_bad_frame = TB, lie_bad_frame = LB)
    X = np.concatenate((X1, X2), axis = 1)
elif mode == 2:
    X, Y = processingTF(TTruth, TLie, combine = False)

print(f'There are {X.shape[0]} batches, each with {X.shape[1]} frames and {X.shape[2]} features. Together there are {X.shape[0] * X.shape[1] * X.shape[2]} data points and {X.shape[0] * X.shape[1]} frames.')

There are 61438 batches, each with 10 frames and 10 features. Together there are 6143800 data points and 614380 frames.


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# no split by person
numOfFrames = 10

TEST_RATIO = 0.2

xTrain, xTest = train_test_split(X, test_size=TEST_RATIO, shuffle=False)
yTrain, yTest = train_test_split(Y, test_size=TEST_RATIO, shuffle=False)

yTrain_temp, yTest_temp = [], []

for i in range(yTrain.shape[0]):
    yTrain_temp.append([1,0]) if yTrain[i] == 0 else yTrain_temp.append([0,1])

for i in range(yTest.shape[0]):
    yTest_temp.append([1,0]) if yTest[i] == 0 else yTest_temp.append([0,1])

y_Train = torch.tensor(yTrain_temp).to(device)
y_Test = torch.tensor(yTest_temp).to(device)

x_Train = torch.tensor(xTrain, dtype=torch.float32).to(device)
x_Test = torch.tensor(xTest, dtype=torch.float32).to(device)

In [6]:
#model prep
featCount = 10
num_frames = 10
encoder_layers = 2

Transformer = classifierTransformer(featCount, encoder_layers, num_frames, device)

# training
def train(model, xTrain, yTrain, xTest, yTest, epochs = 100, lr = 0.005, batch_size = 10):
    """ Train a model on a dataset """
    
    # create a data loader to handle batching
    xTrain_loader = DataLoader(xTrain, batch_size=batch_size, shuffle=False)
    xTest_loader = DataLoader(xTest, batch_size=batch_size, shuffle=False)

    # create a loss function and optimizer
    loss_fn = nn.CrossEntropyLoss()
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    train_losses = []
    test_losses = []
    train_accuracy = []
    test_accuracy = []

    # train the model
    for epoch in range(epochs):

        idx = 0
        
        tot_loss = 0
        tot_acc = 0
        
        for batch in xTrain_loader:

            model.train()

            optimizer.zero_grad()

            # get data
            x_train = batch.to(device).float()
            y_train = torch.tensor(yTrain[idx:min(idx+batch_size,len(yTrain))]).float().clone().detach().to(device)

            # forward pass
            y_pred = model(x_train)

            actual_batch = torch.argmax(y_train, dim=1).long()
            my_pred_batch = torch.argmax(y_pred, dim=1).long()
            tot_acc += ((actual_batch == my_pred_batch).sum().item() / len(actual_batch))
            #print("actual for batch ", idx, " is ", torch.argmax(y_train, dim=1).long())
            #print("my prediction for batch ", idx, " is ", torch.argmax(y_pred, dim=1).long())

            # compute loss
            loss = loss_fn(y_pred,torch.argmax(y_train, dim=1).long())

            tot_loss += loss.item()
            
            # backward pass
            loss.backward()

            # update weights
            optimizer.step()

            idx += batch_size
            
        total_loss = tot_loss / len(xTrain_loader)
        total_acc = tot_acc / len(xTrain_loader)
        train_losses.append(total_loss)
        train_accuracy.append(total_acc)
        print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}, Accuracy: {total_acc:.4f}')

        # evaluate
        model.eval()

        if epoch % 10 == 0:

            with torch.no_grad():
            
                idx_test = 0
                test_acc = 0
                test_loss = 0
                for batch in xTest_loader:
                    xTest = batch.to(device).float()
                    y_test = torch.tensor(yTest[idx_test:min(idx_test+batch_size,len(yTest))]).float().clone().detach().to(device)
                    y_pred = model(xTest)

                    actual_batch = torch.argmax(y_test, dim=1).long()
                    my_pred_batch = torch.argmax(y_pred, dim=1).long()

                    # compute loss
                    loss = loss_fn(y_pred,torch.argmax(y_test, dim=1).long())
                    test_loss += loss.item()

                    #compute test accuracy
                    test_acc += (actual_batch == my_pred_batch).float().mean().item()
                    idx_test += batch_size

                test_acc /= len(xTest_loader)
                test_loss /= len(xTest_loader)

                test_accuracy.append(test_acc)
                test_losses.append(test_loss)

                print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}')



In [7]:
train(Transformer, xTrain, y_Train, xTest, y_Test)

Epoch 1/100, Loss: 0.5628, Accuracy: 0.7004
Test Accuracy: 0.7502
Epoch 2/100, Loss: 0.4839, Accuracy: 0.7465
Epoch 3/100, Loss: 0.4470, Accuracy: 0.7646
Epoch 4/100, Loss: 0.4160, Accuracy: 0.7804
Epoch 5/100, Loss: 0.3937, Accuracy: 0.7938
Epoch 6/100, Loss: 0.3811, Accuracy: 0.8058
Epoch 7/100, Loss: 0.3712, Accuracy: 0.8148
Epoch 8/100, Loss: 0.3555, Accuracy: 0.8245
Epoch 9/100, Loss: 0.3747, Accuracy: 0.8138
Epoch 10/100, Loss: 0.3542, Accuracy: 0.8272
Epoch 11/100, Loss: 0.3391, Accuracy: 0.8365
Test Accuracy: 0.8726
Epoch 12/100, Loss: 0.3402, Accuracy: 0.8369
Epoch 13/100, Loss: 0.3272, Accuracy: 0.8454
Epoch 14/100, Loss: 0.3211, Accuracy: 0.8493
Epoch 15/100, Loss: 0.3505, Accuracy: 0.8332
Epoch 16/100, Loss: 0.3798, Accuracy: 0.8159
Epoch 17/100, Loss: 0.3229, Accuracy: 0.8475
Epoch 18/100, Loss: 0.3317, Accuracy: 0.8410
Epoch 19/100, Loss: 0.4493, Accuracy: 0.7708
Epoch 20/100, Loss: 0.3549, Accuracy: 0.8286
Epoch 21/100, Loss: 0.3620, Accuracy: 0.8278
Test Accuracy: 0.868

KeyboardInterrupt: 

In [ ]:
sweep_config = {
    'method': 'random'
    }
# we can choose from random, grid, and bayes

## The followings are necessary for bayes method
metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    'fc_layer_size': {
        'values': [128, 256, 512]
        },
    }

sweep_config['parameters'] = parameters_dict

# Parameters we don't want to vary
parameters_dict.update({
    'epochs': {'value': 100}
    })

parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
      },
    'batch_size': {
        # integers between 32 and 256
        # with evenly-distributed logarithms 
        'distribution': 'q_log_uniform_values',
        'q': 8,
        'min': 32,
        'max': 256,
      }
    })

import pprint

pprint.pprint(sweep_config)

In [ ]:
import wandb
wandb.login()
print("file exists?", os.path.exists('Classifier.ipynb'))
os.environ["WANDB_NOTEBOOK_NAME"] = "Classifier.ipynb"

def sweep(config=None):
    global counter
    counter+=1
    with wandb.init(config=config, name = f"Experiment{counter}"):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        print(config)
        train(Transformer, xTrain, y_Train, xTest, y_Test, epochs = config.epochs, lr = config.learning_rate, batch_size = config.batch_size)
        wandb.finish()

In [35]:
counter=0
sweep_id = wandb.sweep(sweep_config, project="Classifier_Transformer")
wandb.agent(sweep_id, sweep, count=10)
counter=0